In [6]:
from sentence_transformers import SentenceTransformer

In [5]:
pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
pip install singlestoredb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.1 MB/s eta 0:00:00


In [3]:
from transformers import CLIPTokenizer, CLIPTextModel
import torch
import singlestoredb as s2

In [7]:
import os

from PIL import Image
import numpy as np
from sentence_transformers import SentenceTransformer

PATH = os.getcwd()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu").type


def convert_repo_to_embedding(path:str) -> dict:
  img_model = SentenceTransformer('clip-ViT-B-32', device=DEVICE)
  image_paths = os.listdir(path)

  embeddings = img_model.encode([Image.open(os.path.join(PATH, image_path)) for image_path in image_paths], show_progress_bar=True, device=DEVICE)

  del img_model
  return {image_path: embedding for image_path, embedding in zip(image_paths, embeddings)}

In [8]:
DEVICE

'cpu'

In [9]:
def store_embeddings_db(embeddings: dict) -> None:
    # Create a connection to the database
    conn = s2.connect('admin:<ShpAwYeNOGMAaA7xXN6mzmwf1P1iygxz>@svc-ae8c5252-2f14-4cbf-a2e4-f0531b543234-dml.aws-oregon-3.svc.singlestore.com:3306/')

    with conn:
        with conn.cursor() as cur:
            # Create table if it doesn't exist
            cur.execute('''CREATE TABLE IF NOT EXISTS image_embeddings (
                            image_path VARCHAR(255) PRIMARY KEY,
                            embedding BLOB
                           )''')
            # Insert embeddings
            for image_path, embedding in embeddings.items():
                cur.execute('''INSERT INTO image_embeddings (image_path, embedding)
                               VALUES (%s, %s)
                               ON DUPLICATE KEY UPDATE embedding=VALUES(embedding)''',
                            (image_path, embedding.tobytes()))
            conn.commit()

In [10]:
def get_highest_similarity_image(text: str, top_n: int) -> list:
    conn = s2.connect('admin:<ShpAwYeNOGMAaA7xXN6mzmwf1P1iygxz>@svc-ae8c5252-2f14-4cbf-a2e4-f0531b543234-dml.aws-oregon-3.svc.singlestore.com:3306/')

    with conn:
        with conn.cursor() as cur:
            cur.execute('SELECT image_path, embedding FROM image_embeddings')
            image_embeddings = cur.fetchall()

    text_embed = get_text_embedding(text)
    text_embed = normalize(text_embed)

    similarities = []
    for image_path, embedding in image_embeddings:
        image_embed = np.frombuffer(embedding, dtype=np.float32)
        image_embed = normalize(image_embed)
        similarity_score = get_similarity_score(text_embed, image_embed)
        similarities.append((image_path, similarity_score))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]


In [11]:
normalize = lambda vector: ((vector - vector.mean()) / vector.std())

def get_similarity_score(vector1:np.array, vector2: np.array)-> float:
  return np.dot(normalize(vector1), normalize(vector2))

In [12]:
def get_text_embedding(text):

  model_name = 'openai/clip-vit-base-patch32'
  tokenizer = CLIPTokenizer.from_pretrained(model_name)
  model = CLIPTextModel.from_pretrained(model_name)

  """Generate an embedding for the given text."""
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  with torch.no_grad():
      outputs = model(**inputs)
  # Extract the last hidden state
  embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

  del tokenizer
  del model

  return embedding